In [5]:
from pettingzoo.classic import connect_four_v3
import random
import numpy as np
from tqdm import tqdm
import time

In [6]:

q_table = {}
epsilon=0.1
alpha = 0.1
gamma = 0.6

for _ in tqdm(range(100000)):
    env = connect_four_v3.env()
    env.reset()

    for agent in env.agent_iter():
        observation, reward, termination, truncation, info  = env.last()
        if termination or truncation:
            action = None
            break
        else:
            state=''.join([str(x) for x in observation["observation"].flatten()])
            if state not in q_table.keys():
                q_table[state]=[0 for _ in range(7)]
            if random.uniform(0, 1) < epsilon:
                action = env.action_space(agent).sample(observation["action_mask"]) # Explore action space
            else:
                possible=[q_table[state][i] if observation["action_mask"][i]!=0 else -np.inf for i in range (7)]
                action = np.argmax(possible) # Exploit learned values


        env.step(action)

        observation, reward, termination, truncation, info  = env.last()

        next_state =''.join([str(x) for x in observation["observation"].flatten()])
        if next_state not in q_table.keys():
            q_table[next_state]=[0 for _ in range(7)]

        old_value = q_table[state][action]
        next_max = np.max(q_table[next_state])
            
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)

        q_table[state][action] = new_value

    env.close()

100%|██████████| 100000/100000 [05:18<00:00, 314.34it/s]


In [11]:
env = connect_four_v3.env(render_mode="human")
env.reset()

for agent in env.agent_iter():
    observation, reward, termination, truncation, info  = env.last()
    if termination or truncation:
        action = None
        break
    else:
        state=''.join([str(x) for x in observation["observation"].flatten()])
        if state not in q_table.keys():
            q_table[state]=[0 for _ in range(7)]
        if random.uniform(0, 1) < 0:
            action = env.action_space(agent).sample(observation["action_mask"]) # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        # action = env.action_space(agent).sample(observation["action_mask"])  # this is where you would insert your policy

    env.step(action)

    observation, reward, termination, truncation, info  = env.last()

    next_state =''.join([str(x) for x in observation["observation"].flatten()])
    if next_state not in q_table.keys():
        q_table[next_state]=[0 for _ in range(7)]

    old_value = q_table[state][action]
    next_max = np.max(q_table[next_state])
        
    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)

    q_table[state][action] = new_value
    time.sleep(0.4)
env.close()